# Window LAG

## COVID-19 Data
Notes on the data: This data was assembled based on work done by [Rodrigo Pombo](https://github.com/pomber/covid19) based on [John Hopkins University](https://systems.jhu.edu/research/public-health/ncov/), based on [World Health Organisation](https://www.who.int/health-topics/coronavirus). The data was assembled 21st April 2020 - there are no plans to keep this data set up to date.

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Password? ····


## Window Function
The SQL Window functions include LAG, LEAD, RANK and NTILE. These functions operate over a "window" of rows - typically these are rows in the table that are in some sense adjacent.

In [2]:
covid <- dbReadTable(con, 'covid')
world <- dbReadTable(con, 'world')

## 1. Introducing the `covid` table

The example uses a WHERE clause to show the cases in 'Italy' in March.

```sql
SELECT name, DAY(whn),
 confirmed, deaths, recovered
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

**Modify the query to show data from Spain**

In [3]:
covid %>% 
    filter(name=='Spain' & format(whn, '%m')=='03') %>%
    mutate(day=format(whn, '%d')) %>%
    select(name, day, confirmed, deaths, recovered) %>%
    arrange(day)

name,day,confirmed,deaths,recovered
<chr>,<chr>,<int64>,<int64>,<int64>
Spain,01,84,0,2
Spain,02,120,0,2
Spain,03,165,1,2
Spain,04,222,2,2
Spain,05,259,3,2
Spain,06,400,5,2
Spain,07,500,10,30
Spain,08,673,17,30
Spain,09,1073,28,32


## 2. Introducing the LAG function

The LAG function is used to show data from the preceding row or the table. When lining up rows the data is partitioned by country name and ordered by the data whn. That means that only data from Italy is considered.

```sql
SELECT name, DAY(whn), confirmed,
   LAG(whn, 1) OVER (PARTITION BY name ORDER BY whn)
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

**Modify the query to show confirmed for the day before.**

In [4]:
covid %>%
    mutate(mo=format(whn, '%m'), 
           day=format(whn, '%d')) %>%
    filter(name=='Italy' & mo =='03') %>%
    group_by(name, mo) %>% 
    mutate(lag_cfrm=lag(
        confirmed, default=bit64::NA_integer64_, order_by=whn)) %>%
    mutate(day=format(whn, '%d')) %>%
    ungroup %>%
    select(name, day, confirmed, lag_cfrm)

name,day,confirmed,lag_cfrm
<chr>,<chr>,<int64>,<int64>
Italy,01,1694,NA
Italy,02,2036,1694
Italy,03,2502,2036
Italy,04,3089,2502
Italy,05,3858,3089
Italy,06,4636,3858
Italy,07,5883,4636
Italy,08,7375,5883
Italy,09,9172,7375


### LAG operation

Here is the correct query showing the cases for the day before:

```sql
SELECT name, DAY(whn), confirmed,
   LAG(confirmed, 1) OVER (partition by name ORDER BY whn) AS lag
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

Notice how the values in the LAG column match the value of the row diagonally above and to the left.

name | DAY(whn) | confirmed | dbf
------|---|------|----------
Italy | 1 | **1694** | null
Italy | 2 | 2036 | **1694**
Italy | 3 | 2502 | 2036
Italy | 4 | 3089 | 2502
Italy | 5 | **3858** | 3089
Italy | 6 | 4636 | **3858**
Italy | 7 | 5883 | 4636
Italy | 8 | 7375 | 5883
Italy | 9 | 9172 | 7375
Italy | 10 | 10149 | 9172
... | | |

## 3. Number of new cases

The number of confirmed case is cumulative - but we can use LAG to recover the number of new cases reported for each day.

```sql
SELECT name, DAY(whn), confirmed,
   LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn)
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

**Show the number of new cases for each day, for Italy, for March.**

In [5]:
covid %>% 
    mutate(mo=format(whn, '%m'), 
           day=format(whn, '%d')) %>%
    filter(name=='Italy' & mo=='03') %>%
    group_by(name, mo) %>% 
    mutate(new=confirmed-lag(
        confirmed, default=bit64::NA_integer64_, order_by=whn)) %>%
    ungroup %>%
    select(name, day, new)

name,day,new
<chr>,<chr>,<int64>
Italy,01,NA
Italy,02,342
Italy,03,466
Italy,04,587
Italy,05,769
Italy,06,778
Italy,07,1247
Italy,08,1492
Italy,09,1797


## 4. Weekly changes

The data gathered are necessarily estimates and are inaccurate. However by taking a longer time span we can mitigate some of the effects.

You can filter the data to view only Monday's figures **WHERE WEEKDAY(whn) = 0**.

```sql
SELECT name, DATE_FORMAT(whn,'%Y-%m-%d'), confirmed
 FROM covid
WHERE name = 'Italy'
AND WEEKDAY(whn) = 0
ORDER BY whn
```

**Show the number of new cases in Italy for each week - show Monday only.**

In [6]:
covid %>% 
    mutate(wday=format(whn, '%w')) %>%
    filter(name=='Italy' & wday==0) %>%
    mutate(
        new=confirmed-lag(
            confirmed, default=bit64::NA_integer64_, order_by=whn)) %>%
    select(name, whn, new)

name,whn,new
<chr>,<date>,<int64>
Italy,2020-01-26,NA
Italy,2020-02-02,2
Italy,2020-02-09,1
Italy,2020-02-16,0
Italy,2020-02-23,152
Italy,2020-03-01,1539
Italy,2020-03-08,5681
Italy,2020-03-15,17372
Italy,2020-03-22,34391


## 5. LAG using a JOIN

You can JOIN a table using DATE arithmetic. This will give different results if data is missing.

**Show the number of new cases in Italy for each week - show Monday only.**

In the sample query we JOIN this week tw with last week lw using the DATE_ADD function.

```sql
SELECT tw.name, DATE_FORMAT(tw.whn,'%Y-%m-%d'), 
 tw.confirmed, lw.confirmed
 FROM covid tw LEFT JOIN covid lw ON 
  DATE_ADD(lw.whn, INTERVAL 1 WEEK) = tw.whn
   AND tw.name=lw.name
WHERE tw.name = 'Italy'
ORDER BY tw.whn
```

In [7]:
a <- covid %>% 
    mutate(wday=format(whn, '%w')) %>% 
    filter(name=='Italy' & wday=='0')
a %>% left_join(
    a %>% mutate(
        whn=whn+7, 
        confirmed=bit64::as.character.integer64(confirmed)) %>% 
        select(name, whn, confirmed), 
    by=c(name='name', whn='whn')) %>%
    mutate(new=confirmed.x-confirmed.y) %>%
    select(name, whn, new)

name,whn,new
<chr>,<date>,<int64>
Italy,2020-01-26,NA
Italy,2020-02-02,2
Italy,2020-02-09,1
Italy,2020-02-16,0
Italy,2020-02-23,152
Italy,2020-03-01,1539
Italy,2020-03-08,5681
Italy,2020-03-15,17372
Italy,2020-03-22,34391


## 6. RANK()

The query shown shows the number of confirmed cases together with the world ranking for cases.

United States has the highest number, Spain is number 2...

Notice that while Spain has the second highest confirmed cases, Italy has the second highest number of deaths due to the virus.

```sql
SELECT 
   name,
   confirmed,
   RANK() OVER (ORDER BY confirmed DESC) rc,
   deaths
  FROM covid
WHERE whn = '2020-04-20'
ORDER BY confirmed DESC
```

**Include the ranking for the number of deaths in the table. Only include countries with a population of at least 10 million.**

In [8]:
covid %>%
    inner_join(world %>% 
               select(name, population), by=c(name='name')) %>%
    filter(whn==as.Date('2020-04-20') & population>=1e7) %>%
    mutate(rc=row_number(desc(confirmed)), 
           rc2=row_number(desc(deaths))) %>%
    select(name, confirmed, rc, deaths, rc2) %>%
    arrange(desc(confirmed))

name,confirmed,rc,deaths,rc2
<chr>,<int64>,<int>,<int64>,<int>
United States,784326,1,42094,1
Spain,200210,2,20852,3
Italy,181228,3,24114,2
France,156480,4,20292,4
Germany,147065,5,4862,8
United Kingdom,125856,6,16550,5
Turkey,90980,7,2140,12
China,83817,8,4636,9
Iran,83505,9,5209,7


## 7. Infection rate

The query shown includes a JOIN t the world table so we can access the total population of each country and calculate infection rates (in cases per 100,000).

```sql
SELECT 
   world.name,
   ROUND(100000*confirmed/population,0)
  FROM covid JOIN world ON covid.name=world.name
WHERE whn = '2020-04-20' AND population > 10000000
ORDER BY population DESC
```

**Show the infect rate ranking for each country. Only include countries with a population of at least 10 million.**

In [9]:
covid %>%
    inner_join(world %>% 
               select(name, population), by=c(name='name')) %>%
    filter(whn==as.Date('2020-04-20') & population>=1e7) %>%
    mutate(inf_rate=round(1e5 * confirmed/population, 0)) %>%
    mutate(rank=row_number(inf_rate)) %>%
    arrange(desc(population)) %>%
    select(name, inf_rate, rank)

name,inf_rate,rank
<chr>,<dbl>,<int>
China,6,51
India,1,24
United States,238,87
Indonesia,3,39
Pakistan,4,45
Brazil,19,65
Nigeria,0,13
Bangladesh,2,31
Russia,32,71


## 8. Turning the corner

For each country that has had at last 1000 new cases in a single day, show the date of the peak number of new cases.

In [10]:
covid %>%
    group_by(name) %>%
    mutate(new=confirmed-lag(
        confirmed, default=bit64::NA_integer64_, order_by=whn)) %>%
    mutate(rank=row_number(desc(new))) %>%
    filter(rank==1 & new>=1000) %>%
    select(name, whn, new) %>%
    arrange(whn, name)

name,whn,new
<chr>,<date>,<int64>
China,2020-02-13,15136
Italy,2020-03-21,6557
Switzerland,2020-03-23,1321
Israel,2020-03-25,1131
Spain,2020-03-25,9630
Austria,2020-03-26,1321
Germany,2020-03-27,6933
Iran,2020-03-30,3186
Canada,2020-04-05,2778


In [11]:
dbDisconnect(con)